In [1]:
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x10304b818>>

In [ ]:
class U_DATA:
    def __init__(self, filepath):
        self.filepath = filepath
        self.building_ID = 0
        self.floor = 0
        self.data_num = 0
        self.wifi_loc_time = 0
        self.longitude_list = 0
        self.latitude_list = 0
        self.loc_dict = {}
        self.user_num = 0
        self.user_list = 0
        self.state_count = 0
        self.start_location = np.zeros((2))
        self.total_batch_num = 0
        self.batch_wifi = 0
        self.batch_loc = 0
        self.iter_batch = 0
        self.iter_window = 0
        
    def load_data(self, building_ID, floor):
        self.building_ID = building_ID
        self.floor = floor
        self.data_num = 0
        with open(self.filepath, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (row[523] is not self.building_ID or row[522] is not self.floor):
                    continue
                self.data_num += 1
        
        self.wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open(self.filepath, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (row[523] is not self.building_ID or row[522] is not self.floor):
                    continue
                i = i+1
                if (i > self.data_num):
                    break
                # wifi
                self.wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                self.wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                self.wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                self.wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        self.wifi_loc_time = self.wifi_loc_time[self.wifi_loc_time[:,-1].argsort()]
        self.user_list = np.unique(wifi_loc_time[:, 522])
        self.user_num = len(self.user_list)
        print("There are ", self.user_num, " users.")
        print("The # of total points is ", self.data_num)
        
    def extract_coordinates(self):
        self.longitude_list = sorted(np.unique(self.wifi_loc_time[:, 520]))
        self.latitude_list = sorted(np.unique(self.wifi_loc_time[:, 521]))
        
    def location_dict(self):
        long_dict = {}
        for i in range(len(self.longitude_list)):
            long_dict[float(self.longitude_list[i])] = 0
        for i in range(len(self.latitude_list)):
            self.loc_dict[float(self.latitude_list[i])] = long_dict.copy()

    def catch_user(self, user_index):
        user_walking_num = 0
        for i in range(len(self.wifi_loc_time[:,522])):
            if (self.wifi_loc_time[i, 522] == self.user_list[user_index]):
                user_walking_num += 1
                
        user_fp = np.zeros((user_walking_num, 2+520))
        user_i = 0
        for i in range(len(self.wifi_loc_time[:,522])):
            if (self.wifi_loc_time[i, 522] == 2):
                user_fp[user_i, :2] = np.array(self.wifi_loc_time[i, 520:522])
                user_fp[user_i, 2:] = np.array(self.wifi_loc_time[i, :520])
                user_i += 1
        print("User ", self.user_list[user_index], " who you catch walked through ", user_walking_num, " points")
        
        return user_fp
    
    def user_walking_state(self, user_fp):
        if (self.state_count == 0):
            # find the start
            for i in range(len(self.latitude_list)):
                for j in range(len(self.longitude_list)):
                    if (user_fp[0][0] == self.longitude_list[j] and user_fp[0][1] == self.latitude_list[i]):
                        print(i, j, self.loc_dict[self.latitude_list[i]][self.longitude_list[j]])
                        if (self.loc_dict[self.latitude_list[i]][self.longitude_list[j]] == 0):
                            self.loc_dict[self.latitude_list[i]][self.longitude_list[j]] = self.state_count
                            self.start_location[0] = i
                            self.start_location[1] = j
                            self.state_count = self.state_count + 1
                            print("Start state location:", i, j, self.loc_dict[self.latitude_list[i]][self.longitude_list[j]])
                            break
                               
        for k in range(1,len(user_fp[0])):
            for i in range(len(self.latitude_list)):
                for j in range(len(self.longitude_list)):
                    if (user_fp[k][0] == self.longitude_list[j] and user_fp[k][1] == self.latitude_list[i]):
                        if (self.loc_dict[self.latitude_list[i]][self.longitude_list[j]] == 0 and (i!=65 and j!=11)):
                            self.loc_dict[self.latitude_list[i]][sefl.longitude_list[j]] = self.state_count
                            self.state_count = self.state_count + 1
                        print(k, i, j, self.loc_dict[self.latitude_list[i]][self.longitude_list[j]])
        self.total_batch_num += len(user_fp[0])

    def produce_training_block(self, user_fp, self.seq_window):
        print("This user training contains ", len(user_fp[0]), " blocks and total is ", self.total_batch_num)
        self.batch_wifi = np.zeros((self.total_batch_num, self.seq_window, 520))
        self.batch_loc = np.zeros((self.total_batch_num, self.seq_window), dtype = int)
        
        for k in range(len(user_fp[0])):
            batch_wifi[self.iter_batch][iter_window] = user2_fp[i][2:]
            batch_loc[self.iter_batch,iter_window] = loc_dict[float(user2_fp[i][1])][float(user2_fp[i][0])]
            if (iter_window == (seq_window-1)):
                iter_window = 0
                if (iter_batch < (batch_num-1)):
                    iter_batch = iter_batch + 1
                elif (iter_batch == (batch_num-1)):
                    iter_batch = iter_batch + 1
                    break
            else:
                iter_window = iter_window + 1
        
        
        